In [ ]:
!git clone https://github.com/facebookresearch/demucs.git
%cd demucs
!pip install -e .

In [1]:
%%capture
!apt-get install soundstretch
!apt install ffmpeg

In [2]:
%%capture
!pip install basic-pitch --upgrade

In [61]:
%%capture
%cd /content/
!git clone https://github.com/tatami-galaxy/wav2tab.git
%cd wav2tab/
!python repo_setup.py
!pip install -r requirements.txt

In [62]:
%cd /content/wav2tab/src

/content/wav2tab/src


In [63]:
!python wav2tab_v1.py --in_path /content/thunderstruck.mp3 --out_path /content/

2023-05-10 13:28:08.121393: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-10 13:28:09.154013: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-05-10 13:28:09.154113: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-05-10 13:28:09.154132: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War

In [66]:
!rm -r /content/htdemucs/

In [67]:
!python wav2tab_v1.py --in_path /content/behind_blue_eyes.mp3 --out_path /content/

2023-05-10 13:38:45.360737: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-10 13:38:46.407865: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-05-10 13:38:46.407983: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-05-10 13:38:46.408019: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War

In [59]:
%cd /content/
!rm -r wav2tab

/content


In [ ]:
from argparse import ArgumentParser
import io
from pathlib import Path
import select
from shutil import rmtree
import subprocess as sp
import sys
from typing import Dict, Tuple, Optional, IO
from os.path import dirname, abspath
from basic_pitch.inference import predict
from basic_pitch import ICASSP_2022_MODEL_PATH
import numpy as np
import librosa

# get root directory
#root = abspath(__file__)
#while root.split('/')[-1] != 'wav2tab':
    #root = dirname(root)

# hyps #

model = "htdemucs"
extensions = ["mp3", "wav", "ogg", "flac"]  # we will look for all those file types.
two_stems = None   # only separate one stems from the rest, for instance
# two_stems = "vocals"

# Options for the output audio.
mp3 = True
mp3_rate = 320
float32 = False  # output as float 32 wavs, unsused if 'mp3' is True.
int24 = False    # output as int24 wavs, unused if 'mp3' is True.
# You cannot set both `float32 = True` and `int24 = True` !!

def find_files(in_path):
    out = []
    for file in Path(in_path).iterdir():
        if file.suffix.lower().lstrip(".") in extensions:
            out.append(file)
    return out


def copy_process_streams(process: sp.Popen):
    def raw(stream: Optional[IO[bytes]]) -> IO[bytes]:
        assert stream is not None
        if isinstance(stream, io.BufferedIOBase):
            stream = stream.raw
        return stream

    p_stdout, p_stderr = raw(process.stdout), raw(process.stderr)
    stream_by_fd: Dict[int, Tuple[IO[bytes], io.StringIO, IO[str]]] = {
        p_stdout.fileno(): (p_stdout, sys.stdout),
        p_stderr.fileno(): (p_stderr, sys.stderr),
    }
    fds = list(stream_by_fd.keys())

    while fds:
        # `select` syscall will wait until one of the file descriptors has content.
        ready, _, _ = select.select(fds, [], [])
        for fd in ready:
            p_stream, std = stream_by_fd[fd]
            raw_buf = p_stream.read(2 ** 16)
            if not raw_buf:
                fds.remove(fd)
                continue
            buf = raw_buf.decode()
            std.write(buf)
            std.flush()

# separate all song in directory passed in
def separate_dir(inp=None, outp=None):
    inp = inp 
    outp = outp 
    cmd = ["python3", "-m", "demucs.separate", "-o", str(outp), "-n", model]
    if mp3:
        cmd += ["--mp3", f"--mp3-bitrate={mp3_rate}"]
    if float32:
        cmd += ["--float32"]
    if int24:
        cmd += ["--int24"]
    if two_stems is not None:
        cmd += [f"--two-stems={two_stems}"]
    files = [str(f) for f in find_files(inp)]
    if not files:
        print(f"No valid audio files in {inp}")
        return
    print("Going to separate the files:")
    print('\n'.join(files))
    print("With command: ", " ".join(cmd))
    p = sp.Popen(cmd + files, stdout=sp.PIPE, stderr=sp.PIPE)
    copy_process_streams(p)
    p.wait()
    if p.returncode != 0:
        print("Command failed, something went wrong.")


def separate(inp=None, outp=None):
    inp = inp 
    outp = outp 
    cmd = ["demucs", "-o", str(outp)]
    if mp3:
        cmd += ["--mp3", f"--mp3-bitrate={mp3_rate}"]
    if float32:
        cmd += ["--float32"]
    if int24:
        cmd += ["--int24"]
    if two_stems is not None:
        cmd += [f"--two-stems={two_stems}"]

    print("Going to separate file:")
    print(inp)
    print("With command: ", " ".join(cmd))
    p = sp.Popen(cmd + [inp], stdout=sp.PIPE, stderr=sp.PIPE)

    copy_process_streams(p)
    p.wait()
    if p.returncode != 0:
        print("Command failed, something went wrong.")


def pitch_to_note(note_events):
    ret_list = []
    for event in note_events:
        l = list(event)
        note = librosa.midi_to_note(event[2], unicode=False)  # event[2] is pitch
        l[2] = note
        ret_list.append(l)
    return ret_list

In [ ]:
# song input -> mp3 file (for now)
# make sure file has no space in name (for now)
in_path = '/content/behind_blue_eyes.mp3'
out_path = '/content/'
# stems output
# separate into stems -> other.mp3 has guitar for now
separate(in_path, out_path)

# midi : pretty_midi.PrettyMIDI object
# note_events: A list of note event tuples (start_time_s, end_time_s, pitch_midi, amplitude, bends)
outfile_path = out_path + 'htdemucs/' + in_path.split('/')[-1].split('.mp3')[0] + '/other.mp3'

model_output, midi_data, note_events = predict(outfile_path)

# sort note events according to start time 
sorted_events = sorted(note_events, key=lambda x: x[0])  # x[0] is start time

# midi pitch to note
# each event is now a list [start_time_s, end_time_s, pitch_midi, amplitude, bends]
note_events = pitch_to_note(sorted_events)

#print(note_events[0])

Going to separate file:
/content/behind_blue_eyes.mp3
With command:  demucs -o /content/ --mp3 --mp3-bitrate=320


Downloading: "https://dl.fbaipublicfiles.com/demucs/hybrid_transformer/955717e8-8726e21a.th" to /root/.cache/torch/hub/checkpoints/955717e8-8726e21a.th


Important: the default model was recently changed to `htdemucs` the latest Hybrid Transformer Demucs model. In some cases, this model can actually perform worse than previous models. To get back the old default model use `-n mdx_extra_q`.


100%|██████████| 80.2M/80.2M [00:00<00:00, 143MB/s]


Selected model is a bag of 1 models. You will see that many progress bars per track.
Separated tracks will be stored in /content/htdemucs
Separating track /content/behind_blue_eyes.mp3


100%|██████████████████████████████████████████████| 368.54999999999995/368.54999999999995 [00:20<00:00, 18.16seconds/s]


Predicting MIDI for /content/htdemucs/behind_blue_eyes/other.mp3...


In [ ]:
print(len(note_events))
print(note_events[0])

3721

In [ ]:
import pickle
with open("/content/note_events", "wb") as fp:
    pickle.dump(note_events, fp)

NameError: ignored

In [20]:
fretboard ={
    'E4': {1: 0, 2: 5, 3: 9, 4: -1, 5: -1, 6: -1},
    'F4': {1: 1, 2: 6, 3: 10, 4: -1, 5: -1, 6: -1},
    'F#4': {1: 2, 2: 7, 3: 11, 4: -1, 5: -1, 6: -1},
    'G4': {1: 3, 2: 8, 3: 12, 4: -1, 5: -1, 6: -1},
    'G#4': {1: 4, 2: 9, 3: -1, 4: -1, 5: -1, 6: -1},
    'A4': {1: 5, 2: 10, 3: -1, 4: -1, 5: -1, 6: -1},
    'A#4': {1: 6, 2: 11, 3: -1, 4: -1, 5: -1, 6: -1},
    'B4': {1: 7, 2: 12, 3: -1, 4: -1, 5: -1, 6: -1},
    'C5': {1: 8, 2: -1, 3: -1, 4: -1, 5: -1, 6: -1},
    'C#5': {1: 9, 2: -1, 3: -1, 4: -1, 5: -1, 6: -1},
    'D5': {1: 10, 2: -1, 3: -1, 4: -1, 5: -1, 6: -1},
    'D#5': {1: 11, 2: -1, 3: -1, 4: -1, 5: -1, 6: -1},
    'E5': {1: 12, 2: -1, 3: -1, 4: -1, 5: -1, 6: -1},
    'B3': {1: -1, 2: 0, 3: 4, 4: 9, 5: -1, 6: -1},
    'C4': {1: -1, 2: 1, 3: 5, 4: 10, 5: -1, 6: -1},
    'C#4': {1: -1, 2: 2, 3: 6, 4: 11, 5: -1, 6: -1},
    'D4': {1: -1, 2: 3, 3: 7, 4: 12, 5: -1, 6: -1},
    'D#4': {1: -1, 2: 4, 3: 8, 4: -1, 5: -1, 6: -1},
    'G3': {1: -1, 2: -1, 3: 0, 4: 5, 5: 10, 6: -1},
    'G#3': {1: -1, 2: -1, 3: 1, 4: 6, 5: 11, 6: -1},
    'A3': {1: -1, 2: -1, 3: 2, 4: 7, 5: 12, 6: -1},
    'A#3': {1: -1, 2: -1, 3: 3, 4: 8, 5: -1, 6: -1},
    'D3': {1: -1, 2: -1, 3: -1, 4: 0, 5: 5, 6: 10},
    'D#3': {1: -1, 2: -1, 3: -1, 4: 1, 5: 6, 6: 11},
    'E3': {1: -1, 2: -1, 3: -1, 4: 2, 5: 7, 6: 12},
    'F3': {1: -1, 2: -1, 3: -1, 4: 3, 5: 8, 6: -1},
    'F#3': {1: -1, 2: -1, 3: -1, 4: 4, 5: 9, 6: -1},
    'A2': {1: -1, 2: -1, 3: -1, 4: -1, 5: 0, 6: 5},
    'A#2': {1: -1, 2: -1, 3: -1, 4: -1, 5: 1, 6: 6},
    'B2': {1: -1, 2: -1, 3: -1, 4: -1, 5: 2, 6: 7},
    'C3': {1: -1, 2: -1, 3: -1, 4: -1, 5: 3, 6: 8},
    'C#3': {1: -1, 2: -1, 3: -1, 4: -1, 5: 4, 6: 9},
    'E2': {1: -1, 2: -1, 3: -1, 4: -1, 5: -1, 6: 0},
    'F2': {1: -1, 2: -1, 3: -1, 4: -1, 5: -1, 6: 1},
    'F#2': {1: -1, 2: -1, 3: -1, 4: -1, 5: -1, 6: 2},
    'G2': {1: -1, 2: -1, 3: -1, 4: -1, 5: -1, 6: 3},
    'G#2': {1: -1, 2: -1, 3: -1, 4: -1, 5: -1, 6: 4},
}

In [21]:
import pickle
%cd /content/
with open("note_events", "rb") as fp:
    note_events = pickle.load(fp)

print(len(note_events))

/content
3721


In [ ]:
#diffs = []
#for i in range(len(note_events) - 1):
    #diffs.append(note_events[i+1][0] - note_events[i][0])

#len([x for x in diffs if x < 3])

In [ ]:
for i in range(10):
    print(note_events[i])

[0.5108390022675737, 0.7894784580498866, 'E2', 0.6735251, [1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]
[0.5224489795918368, 2.4161591836734693, 'E1', 0.43101585, [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]
[0.7894784580498866, 1.2887074829931973, 'E2', 0.6283504, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]
[0.8010884353741496, 2.2420095238095237, 'B2', 0.46589294, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
# take first choice #

tab = []

E1 = 'E'+'-'*50
B = 'B'+'-'*50
G = 'G'+'-'*50
D = 'D'+'-'*50
A = 'A'+'-'*50
E2 = 'E'+'-'*50

tab.append(E1)
tab.append(B)
tab.append(G)
tab.append(D)
tab.append(A)
tab.append(E2)

for i in range(40):
    note = note_events[i][2]
    if note not in fretboard: continue
    for string, fret in fretboard[note].items():
        if fret != -1:
            new_string = list(tab[string-1])
            new_string[i+2] = str(fret)
            tab[string-1] = ''.join(new_string)
            break

            






# final print
for string in tab:
    print(string)

E-----4----40--3---------------0------5------------
B------0------0--0-0----0-------3------------------
G-------0-------0-------------------0--------------
D--------2---2--------------0----0-0---0-0---------
A----2------------2-2----2-------------------------
E-0-0-----0----------3-3---3-33---3--3--3----------


In [ ]:
note_events[1]

[0.626938775510204,
 0.7662585034013606,
 'D#4',
 0.65871483,
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]

In [ ]:
new_string = list(tab[0])
new_string[2] = '2'
tab[0] = ''.join(new_string)

for string in tab:
    print(string)

E-2------------------------------------------------
B--------------------------------------------------
G--------------------------------------------------
D--------------------------------------------------
A--------------------------------------------------
E--------------------------------------------------


In [57]:
class GuitarCMD:

    def __init__(self) -> None:

        self.strings = []
        self.tab_length = 100
        self.make_strings()


    def make_strings(self):

        E1 = 'E'+'-'*self.tab_length
        B = 'B'+'-'*self.tab_length
        G = 'G'+'-'*self.tab_length
        D = 'D'+'-'*self.tab_length
        A = 'A'+'-'*self.tab_length
        E2 = 'E'+'-'*self.tab_length

        self.strings.append(E1)
        self.strings.append(B)
        self.strings.append(G)
        self.strings.append(D)
        self.strings.append(A)
        self.strings.append(E2)


    def generate(self, note_events):
        end_space = 5
        gap = 0
        #for i in range(self.tab_length - end_space):
        for i in range(50):
            note = note_events[i][2]  # (start_time_s, end_time_s, pitch_midi, amplitude, bends)
            if note not in fretboard: continue
            for string, fret in fretboard[note].items():
                if fret != -1:
                    new_string = list(self.strings[string-1])
                    new_string[i+2+gap] = str(fret)
                    self.strings[string-1] = ''.join(new_string) 
                    break
            gap += 1

        # print tab to cmd
        for string in self.strings:
            print(string)

In [58]:
tab_generator = GuitarCMD()
tab_generator.generate(note_events)

E--------4---------4-0-----3-----------------------------0-------------5----------------------0------
B----------0-------------0-----0---0--------0--------------3------------------------3---3------------
G------------0---------------0-------------------------------------0--------------2-------2----------
D--------------2-------2---------------------------0---------0---0-------0---0-----------------------
A------2-------------------------2---2--------2---------------------------------------0-----0--------
E-0--0-----------0---------------------3--3------3---3-3-------3-----3-----3-------------------------
